# Importing Data

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import datasets as ds

# Preparing data

In [2]:
(X_train, y_train), (X_test, y_test) = ds.cifar10.load_data()
classes = ["airplane","automobile","bird","cat","deer","dog","frog","horse","ship","truck"]
y_train2 = np.ndarray(shape=(50000, 10))
y_test2 = np.ndarray(shape=(10000, 10))
for row in range(len(y_train)):
    temp = y_train[row, 0]
    rowToAdd = np.ndarray(shape=(1,10))
    for var in range(10):
        rowToAdd[0, var] = 0
        if temp == var:
            rowToAdd[0, var] = 1
    y_train2[row,] = rowToAdd
for row in range(len(y_test)):
    temp = y_test[row, 0]
    rowToAdd = np.ndarray(shape=(1,10))
    for var in range(10):
        rowToAdd[0, var] = 0
        if temp == var:
            rowToAdd[0, var] = 1
    y_test2[row,] = rowToAdd


# Training model

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator

with tf.device('/gpu:0'):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
        
#     model.add(Conv2D(64, (3, 3), activation='relu'))
#     model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Dropout(0.2))
    model.add(Flatten())
    
#     model.add(Dense(64, activation='relu'))
    model.add(Dense(10, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='rmsprop',
                  metrics=['accuracy'])
    datagen = ImageDataGenerator(
        rescale=1./255,
        featurewise_center=True,
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode = 'nearest')
    datagen.fit(X_train)
    # fits the model on batches with real-time data augmentation:
    model.fit(datagen.flow(X_train, y_train2, batch_size=32),
            steps_per_epoch=len(X_train) / 32, epochs=50)
    print(model.summary())

Epoch 1/50
1562/1562 [==============================] - 60s 38ms/step - loss: 1.8352 - accuracy: 0.3232: 0s - loss: 1.8359 - accura
Epoch 2/50
1562/1562 [==============================] - 62s 39ms/step - loss: 1.4630 - accuracy: 0.4720
Epoch 3/50
1562/1562 [==============================] - 60s 38ms/step - loss: 1.3585 - accuracy: 0.5193
Epoch 4/50
1562/1562 [==============================] - 60s 39ms/step - loss: 1.3036 - accuracy: 0.5333
Epoch 5/50
1562/1562 [==============================] - 62s 40ms/step - loss: 1.2456 - accuracy: 0.5572
Epoch 6/50
1562/1562 [==============================] - 59s 38ms/step - loss: 1.2260 - accuracy: 0.5689
Epoch 7/50
1562/1562 [==============================] - 51s 33ms/step - loss: 1.1943 - accuracy: 0.5829
Epoch 8/50
1562/1562 [==============================] - 58s 37ms/step - loss: 1.1884 - accuracy: 0.58360s - loss: 1.1885 - accu
Epoch 9/50
1562/1562 [==============================] - 57s 36ms/step - loss: 1.1580 - accuracy: 0.5991
Epoch 10/50


In [ ]:
import sklearn
from sklearn.metrics import accuracy_score
y_pred = model.predict(X_test)
print("accuracy: " + sklearn.metrics.accuracy_score(y_pred, y_test))
modelFileName = '../Models/cifar_images.h5'
model.save(modelFileName)
#del model  # deletes the existing model variable
print('model saved as', modelFileName)